In [1]:
test_ds_worker_name = "g_t_apn"  #<- force DL 
model_worker_name = test_ds_worker_name


!pip uninstall -y fashion_nets
!pip uninstall -y fashion_datasets

In [2]:
#@title Pip Installs & Imports
#!pip uninstall -y fashion_nets
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionDatasets.git
!pip install -q git+https://github.com/NiklasHoltmeyer/FashionNets.git

try:
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
except:
  from fashionnets.train_jobs.loader.job_loader import prepare_environment, load_job_settings, history_to_csv_string
  from fashionnets.train_jobs.loader.model_loader import load_siamese_model_from_train_job
from fashionnets.train_jobs.loader.checkpoint_loader import load_latest_checkpoint

from fashionnets.models.layer.Augmentation import compose_augmentations
  

     |████████████████████████████████| 958 kB 15.4 MB/s 
     |████████████████████████████████| 631 kB 31.9 MB/s 
     |████████████████████████████████| 138 kB 45.4 MB/s 
     |████████████████████████████████| 356 kB 47.5 MB/s 
     |████████████████████████████████| 138 kB 49.0 MB/s 
     |████████████████████████████████| 138 kB 46.2 MB/s 
     |████████████████████████████████| 153 kB 48.1 MB/s 
     |████████████████████████████████| 137 kB 47.5 MB/s 
     |████████████████████████████████| 136 kB 52.4 MB/s 
     |████████████████████████████████| 136 kB 51.0 MB/s 
     |████████████████████████████████| 136 kB 52.6 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 
     |████████████████████████████████| 48 kB 4.5 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 40 kB 5.2 MB/s 
     |████████████████████████████████| 354 kB 43.1 MB/s 
     |██████████████████

In [ ]:
#@title Prepare Environment. Download Datasets
from fashionnets.train_jobs.loader.job_loader import add_back_bone_to_train_job

environment, training_job_cfg = prepare_environment(test_ds_worker_name, debugging=False)
import kaggle #<- Requires Secrets, therefore prepare_environment needs to be run before
kaggle.api.authenticate()

kaggle_downloader = kaggle.api.dataset_download_files #<- sadly needs to be injected

train_job = load_job_settings(environment=environment, training_job_cfg=training_job_cfg, kaggle_downloader=kaggle_downloader)

job_settings = add_back_bone_to_train_job(environment=environment, **training_job_cfg)

def pass_(a, b, unzip):
  return
kaggle_downloader = pass_

environment_m, training_job_cfg_m = prepare_environment(model_worker_name, debugging=False)
train_job_m = load_job_settings(environment=environment_m, training_job_cfg=training_job_cfg_m, kaggle_downloader=kaggle_downloader)
job_settings_m = add_back_bone_to_train_job(environment=environment_m, **training_job_cfg)


[2021-12-16 17:56:53,036] {Environment.py:25} DEBUG - Environment: google
Mounted at /gdrive/
[2021-12-16 17:57:15,371] {Environment.py:69} DEBUG - Download: masterokay/deep-fashion-1-256


In [ ]:
#@title Load Modell from Remote CBIR
from fashionnets.evaluate.EvaluateDeepFashion import Evaluate
from fashiondatasets.deepfashion1.DeepFashion1CBIR import DeepFashion1CBIR
from fashionnets.evaluate.helper.evaluate_helper_loader import load_model
from fashionnets.evaluate.helper.evaluate_helper_loader import load_dataset

from fashionnets.train_jobs.loader.path_loader import _load_embedding_base_path, _load_dataset_base_path
from fashionnets.models.layer.Augmentation import compose_augmentations

siamese_model, init_epoch, _callbacks = load_siamese_model_from_train_job(force_load_weights=True, load_weights=True, **train_job_m)
print("init_epoch", init_epoch)
print("Callbacks", _callbacks)

base_path = _load_dataset_base_path(**job_settings)
#base_path = base_path.replace("own_256", "deep_fashion_1_256")
embedding_base_path = _load_embedding_base_path(**job_settings)
augmentation = compose_augmentations()(False)

df_cbir = DeepFashion1CBIR(base_path=base_path, 
                 model = siamese_model.siamese_network.feature_extractor,
                 embedding_path=embedding_base_path, 
                 augmentation=augmentation,
                 image_suffix="_256",
                 batch_size=256)
zip_path = df_cbir.bulk_embed(zip_=True)
embedding_name = f"emb_{job_settings_m['run_name']}_{init_epoch-1}.zip"

!mv $zip_path /gdrive/MyDrive/results/$embedding_name

In [ ]:
#@title load all 4 Modells from Train-Job-CP_Path

try:
  del siamese_model
except:
  pass

cp_path = train_job_m["path"]["checkpoint"]#"./"
modelle = load_model(cp_path, train_job)
back_bone_modell = modelle["quadtruplet"].siamese_network.feature_extractor

datasets = load_dataset(model=back_bone_modell, job_settings=job_settings)

assert sorted(datasets.keys()) == sorted(modelle.keys())

In [ ]:
from tqdm.auto import tqdm 

steps = None
losses = list(modelle.keys())
losses = list(reversed(losses))
results = {}

print({"name": job_settings_m['run_name'], "epoch": init_epoch-1, "loss": results})

for loss_fn_str in tqdm(losses, desc="Calc Losses"):
  print(loss_fn_str)
  model = modelle[loss_fn_str]
  loss_dataset = datasets[loss_fn_str]

  results[loss_fn_str] = {}

  for ds_name, ds in loss_dataset.items():
    print(ds_name)
    loss_ =model.evaluate(ds, steps=steps)
    results[loss_fn_str][ds_name] = loss_
    print(results)

    print("***")
    print({"name": job_settings_m['run_name'], "epoch": init_epoch-1, "loss": results})
    print("***")

In [ ]:
print(results)

In [ ]:
results

In [ ]:
f_data = {
    "name": job_settings_m['run_name'],
    "epoch": init_epoch-1,
    "loss": results
}

f_data


In [ ]:
print(f_data)